In [1]:
!pip install numpy gymnasium wandb matplotlib

In [2]:
import numpy as np
import gymnasium as gym
import wandb
import random
import matplotlib.pyplot as plt
from collections import defaultdict

# Set random seed for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    gym.utils.seeding.np_random(seed)
    return seed

# Discretize the continuous state space
class StateDiscretizer:
    def __init__(self, n_bins=10):
        self.n_bins = n_bins
        # CartPole-v1 state space bounds
        self.cart_position_bins = np.linspace(-2.4, 2.4, n_bins)
        self.cart_velocity_bins = np.linspace(-4, 4, n_bins)
        self.pole_angle_bins = np.linspace(-0.2095, 0.2095, n_bins)
        self.pole_velocity_bins = np.linspace(-4, 4, n_bins)

    def discretize(self, state):
        cart_pos, cart_vel, pole_angle, pole_vel = state

        # Discretize each dimension
        cart_pos_bin = np.digitize(cart_pos, self.cart_position_bins)
        cart_vel_bin = np.digitize(cart_vel, self.cart_velocity_bins)
        pole_angle_bin = np.digitize(pole_angle, self.pole_angle_bins)
        pole_vel_bin = np.digitize(pole_vel, self.pole_velocity_bins)

        # Combine into a single discrete state
        return (cart_pos_bin, cart_vel_bin, pole_angle_bin, pole_vel_bin)

# SARSA Agent with epsilon-greedy exploration
class SARSAAgent:
    def __init__(self, state_discretizer, action_space, alpha=0.1, gamma=0.99, epsilon=0.1):
        self.state_discretizer = state_discretizer
        self.action_space = action_space
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = defaultdict(lambda: np.zeros(action_space.n))

    def select_action(self, state):
        state = self.state_discretizer.discretize(state)

        # Epsilon-greedy policy
        if np.random.random() < self.epsilon:
            return self.action_space.sample()  # Explore
        else:
            return np.argmax(self.q_table[state])  # Exploit

    def update(self, state, action, reward, next_state, next_action, done):
        state = self.state_discretizer.discretize(state)
        next_state = self.state_discretizer.discretize(next_state)

        # SARSA update rule
        current_q = self.q_table[state][action]

        if done:
            target_q = reward
        else:
            target_q = reward + self.gamma * self.q_table[next_state][next_action]

        # Update Q-value
        self.q_table[state][action] += self.alpha * (target_q - current_q)

# Q-Learning Agent with Softmax exploration
class QLearningAgent:
    def __init__(self, state_discretizer, action_space, alpha=0.1, gamma=0.99, temperature=1.0):
        self.state_discretizer = state_discretizer
        self.action_space = action_space
        self.alpha = alpha
        self.gamma = gamma
        self.temperature = temperature
        self.q_table = defaultdict(lambda: np.zeros(action_space.n))

    def select_action(self, state):
        state = self.state_discretizer.discretize(state)

        # Softmax exploration
        q_values = self.q_table[state]
        exp_q = np.exp(q_values / self.temperature)
        probabilities = exp_q / np.sum(exp_q)

        # Sample action based on probabilities
        return np.random.choice(self.action_space.n, p=probabilities)

    def update(self, state, action, reward, next_state, done):
        state = self.state_discretizer.discretize(state)
        next_state = self.state_discretizer.discretize(next_state)

        # Q-Learning update rule
        current_q = self.q_table[state][action]

        if done:
            target_q = reward
        else:
            target_q = reward + self.gamma * np.max(self.q_table[next_state])

        # Update Q-value
        self.q_table[state][action] += self.alpha * (target_q - current_q)

# Training function for SARSA
def train_sarsa(config=None):
    with wandb.init(config=config):
        config = wandb.config

        # Fixed seeds
        seeds = [42, 123, 456, 789, 101]
        all_final_rewards = []

        for seed in seeds:
            # Set environment
            env = gym.make("CartPole-v1")

            # Set random seed
            set_seed(seed)

            # Initialize state discretizer
            state_discretizer = StateDiscretizer(n_bins=config.n_bins)

            # Initialize SARSA agent
            agent = SARSAAgent(
                state_discretizer=state_discretizer,
                action_space=env.action_space,
                alpha=config.alpha,
                gamma=config.gamma,
                epsilon=config.epsilon
            )

            # Training loop
            episode_rewards = []

            for episode in range(config.n_episodes):
                state, _ = env.reset(seed=seed + episode)
                action = agent.select_action(state)
                done = False
                episode_reward = 0

                while not done:
                    next_state, reward, terminated, truncated, _ = env.step(action)
                    done = terminated or truncated
                    episode_reward += reward

                    # Select next action using current policy
                    next_action = agent.select_action(next_state)

                    # Update Q-values
                    agent.update(state, action, reward, next_state, next_action, done)

                    # Move to next state and action
                    state = next_state
                    action = next_action

                episode_rewards.append(episode_reward)

                # Log individual seed performance
                wandb.log({
                    f"seed_{seed}_episode": episode,
                    f"seed_{seed}_reward": episode_reward,
                })

            # Store final performance (avg of last 50 episodes)
            final_reward = np.mean(episode_rewards[-50:])
            all_final_rewards.append(final_reward)
            env.close()

        # Calculate average performance across all seeds
        avg_final_reward = np.mean(all_final_rewards)

        # Log the average performance
        wandb.log({
            "average_reward_across_seeds": avg_final_reward
        })

        return avg_final_reward

# Training function for Q-Learning
# Modified training function for Q-Learning
def train_qlearning(config=None):
    with wandb.init(config=config):
        config = wandb.config

        # Fixed seeds
        seeds = [42, 123, 456, 789, 101]
        all_final_rewards = []

        for seed in seeds:
            # Set environment
            env = gym.make("CartPole-v1")

            # Set random seed
            set_seed(seed)

            # Initialize state discretizer
            state_discretizer = StateDiscretizer(n_bins=config.n_bins)

            # Initialize Q-Learning agent
            agent = QLearningAgent(
                state_discretizer=state_discretizer,
                action_space=env.action_space,
                alpha=config.alpha,
                gamma=config.gamma,
                temperature=config.temperature
            )

            # Training loop
            episode_rewards = []

            for episode in range(config.n_episodes):
                state, _ = env.reset(seed=seed + episode)
                done = False
                episode_reward = 0

                while not done:
                    action = agent.select_action(state)
                    next_state, reward, terminated, truncated, _ = env.step(action)
                    done = terminated or truncated
                    episode_reward += reward

                    # Update Q-values
                    agent.update(state, action, reward, next_state, done)

                    # Move to next state
                    state = next_state

                episode_rewards.append(episode_reward)

                # Log individual seed performance
                wandb.log({
                    f"seed_{seed}_episode": episode,
                    f"seed_{seed}_reward": episode_reward,
                })

            # Store final performance (avg of last 50 episodes)
            final_reward = np.mean(episode_rewards[-50:])
            all_final_rewards.append(final_reward)
            env.close()

        # Calculate average performance across all seeds
        avg_final_reward = np.mean(all_final_rewards)

        # Log the average performance
        wandb.log({
            "average_reward_across_seeds": avg_final_reward
        })

        return avg_final_reward


# Sweep configuration for SARSA
# Updated sweep configuration for SARSA
sarsa_sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'average_reward_across_seeds',
        'goal': 'maximize'
    },
    'parameters': {
        'alpha': {
            'min': 0.01,
            'max': 0.5
        },
        'gamma': {
            'value': 0.99
        },
        'epsilon': {
            'min': 0.01,
            'max': 0.3
        },
        'n_bins': {
            'values': [10, 15, 20]
        },
        'n_episodes': {
            'value': 1000
        }
    }
}

# Sweep configuration for Q-Learning
qlearning_sweep_config = {
    'method': 'bayes',  # Bayesian optimization
    'metric': {
        'name': 'average_reward',
        'goal': 'maximize'
    },
    'parameters': {
        'alpha': {
            'min': 0.01,
            'max': 0.5
        },
        'gamma': {
            'value': 0.99  # Fixed as recommended
        },
        'temperature': {
            'min': 0.1,
            'max': 2.0
        },
        'n_bins': {
            'values': [10, 15, 20]
        },
        'n_episodes': {
            'value': 1000
        },
        'seed': {
            'values': [42, 123, 456, 789, 101]  # 5 random seeds
        }
    }
}







In [3]:
# Initialize and run the SARSA sweep
def run_sarsa_sweep():
    sweep_id = wandb.sweep(sarsa_sweep_config, project="RL_CartPole_SARSA")
    wandb.agent(sweep_id, function=train_sarsa, count=50)

# Initialize and run the Q-Learning sweep
def run_qlearning_sweep():
    sweep_id = wandb.sweep(qlearning_sweep_config, project="RL_CartPole_QLearning")
    wandb.agent(sweep_id, function=train_qlearning, count=50)
# Run the sweeps
if __name__ == "__main__":
    # Uncomment one of these to run the respective sweep
    run_sarsa_sweep()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

average_reward_across_seeds,▁
seed_101_episode,▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇██
seed_101_reward,▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▄▁▂▅▁▁▄▁▂▄▁▃▃▃▄▄▃▁█▂▃▆▄
seed_123_episode,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▆▆▇▇▇▇▇▇█████
seed_123_reward,▁▂▃▂▃▃▂▅▃▃▃▃▂▄▃▃▂▃▂▅▁▃▃▃▅▅▇▅▅▆▅▃▃▅▅█▅▅▇▇
seed_42_episode,▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▆▆▇▇▇▇▇▇████
seed_42_reward,▁▁▁▁▁▂▁▁▁▂▂▁▂▁▄▁▁▁▂▂▄▁▃▃▁▂▂▁▃▃▁▄▇▁▅▃█▂▃▃
seed_456_episode,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇████
seed_456_reward,▂█▂▁▂▁▂▁▁▁▂▂▂▁▂▃▁▂▂▁▄▁▃▄▁▃▂▂▄▂▂▄▂▃▂▂▃▃▅▃
seed_789_episode,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇██
seed_789_reward,▁▁▁▂▁▂▁▁▂▁▁▁▅▂▂▁▁▁▂▂▃▃▂▅▂█▂▁▅▄▂▇▅▂▃▂█▆▅▃


wandb: Agent Starting Run: z89sxp0k with config:
wandb: 	alpha: 0.27964880484316645
wandb: 	epsilon: 0.027240058365611223
wandb: 	gamma: 0.99
wandb: 	n_bins: 15
wandb: 	n_episodes: 1000


average_reward_across_seeds,▁
seed_101_episode,▁▁▁▁▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇█████
seed_101_reward,▂▂▁▂▂▂▁▁▂▂▁▂▂▂▁▁▂▁▁▂▂▁▂▁▁▁▁▂▁▁▂▁▁▂▂▂█▂▁▂
seed_123_episode,▁▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
seed_123_reward,▅▁▁▁▅▃▅▃▃▃▅▅█▃▅▅▃▅▅▅▅▅▅▆▆▅▅▁▆▆▃▃▆▆▁▁▆▅▆▆
seed_42_episode,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇█
seed_42_reward,▁▁▁▁▂▁▂▁▁▂▁▁▁▂▂▂▂▂▁▃▂▂▂▂▁▂▂▂▁▁▁▁▁▂▁▂▁▂▂█
seed_456_episode,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇█████
seed_456_reward,▁▁▂▂▁▃▁▁▁▂▁▁▁▁▁▁▁█▁▁▁▁▁▂▁▁▁▁▁▂▃▁▁▁▁▂▁▂▃▂
seed_789_episode,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇██
seed_789_reward,▁▂▁▂▁▂▂▁▂▁▂▂▂▂▇▁▂█▂▂▂▁▂▂▃▂▂▂▁▂▂▂▄▃▂▇▂▃▄▇


wandb: Agent Starting Run: gvoxqbps with config:
wandb: 	alpha: 0.4264348091896062
wandb: 	epsilon: 0.28660574958994933
wandb: 	gamma: 0.99
wandb: 	n_bins: 20
wandb: 	n_episodes: 1000


average_reward_across_seeds,▁
seed_101_episode,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇███
seed_101_reward,▁▁▂▁▁▂▃▁▃▆▆▄▂▆▂▃▃▄▅▂█▇▄▂▄▄▄▃▅▃▃▃▅▁▅▃▃▂▅▃
seed_123_episode,▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇██
seed_123_reward,▂▂▁▁▁▁▁▁▁▂▃▄▃▄▃▂▄▃▄▇▃▄▂▄▄▄▃▄▅█▂▃▄▂▂▃▂▂▄▄
seed_42_episode,▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇████
seed_42_reward,▁▁▁▁▁▁▁▂▂▃▄▃█▁▂▇▂▄▂▄▃▃▅▂▂▇▄▆▃▅▄▄▄▄▄▆▅▅▃▆
seed_456_episode,▁▁▁▂▂▂▂▃▃▃▄▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇█
seed_456_reward,▁▁▂▁▁▂▁▂▄▂▂▄▂▃▃▄▁▄▃▂▃▂▃▅▂▃▆▃▃▄▄█▄▄▃▇▄▂▃▂
seed_789_episode,▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇██
seed_789_reward,▁▁▁▁▁▁▁▁▁▃▃▂▁▃▅▃▄▃█▃▃▅▂▃▂▂▃▃▄▅▄▃▆▄▅▄▆▂▄▄


In [4]:
run_qlearning_sweep()

Create sweep with ID: kreeqo81
Sweep URL: https://wandb.ai/abhijithvinod-indian-institue-of-technology-madras/RL_CartPole_QLearning/sweeps/kreeqo81


wandb: Agent Starting Run: nep5lbdk with config:
wandb: 	alpha: 0.06738119229195272
wandb: 	gamma: 0.99
wandb: 	n_bins: 20
wandb: 	n_episodes: 1000
wandb: 	seed: 123
wandb: 	temperature: 1.8862275116226144


average_reward_across_seeds,▁
seed_101_episode,▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇██
seed_101_reward,▂▂▁▁▁▂▃▁▁▁▄▃▂▃▁▁▂▃▂▆▄▃▃▃▅▃▆▅▄█▇▆▆▆▅▅▆▄█▅
seed_123_episode,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇█████
seed_123_reward,▁▁▂▁▂▁▁▁▁▂▂▁▁▂▂▂▂▃▃▃▄▂▄▃▂▄█▄▄▃▆▆▃▇▄▄▄▅▃▄
seed_42_episode,▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇▇▇▇█████
seed_42_reward,▂▃▂▂▁▁▂▁▁▁▃▃▆▃▃▆▄▃▅▃▅▄▇▆▆▇█▆▄▆▇█▅█▇▆▃▇█▆
seed_456_episode,▁▁▁▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
seed_456_reward,▁▁▁▁▁▃▁▂▃▃▁▂▂▂▁▂▁▁▃▄▄▁▇▇▃▃▄▆▄█▆▃▅▅▅▅▇▅▆▅
seed_789_episode,▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇███
seed_789_reward,▁▃▁▂▂▃▄▁▃▁▂▄▃▃▄▂▃▅▄▅▄▂▄▄▆▃█▄▇▅▅▃▄▅▅▅▇▄▅▇


wandb: Agent Starting Run: dcuhpydd with config:
wandb: 	alpha: 0.0766772249504152
wandb: 	gamma: 0.99
wandb: 	n_bins: 20
wandb: 	n_episodes: 1000
wandb: 	seed: 101
wandb: 	temperature: 1.2610958209564451


average_reward_across_seeds,▁
seed_101_episode,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▇▇▇▇▇██
seed_101_reward,▂▁▂▁▁▂▂▂▁▁▂▂▂▁▂▃▁▄▄▃▃▅▆▄▅▅▃▅▄▆▅▆▆█▂▇▄▆▅▆
seed_123_episode,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇█████
seed_123_reward,▁▁▂▂▃▂▁▁▃▂▁▂▁▃▂▆▆▂▅▁▂▆▆▅▇▃▇▄▇▆▃▆▅▄█▃▇▄▂█
seed_42_episode,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇██
seed_42_reward,▁▁▁▁▂▃▂▅▁▁▄▄▁▂▅▆▃▆▆▅▆▆▂▆▁▅▆▃▄▃▅▇▆█▃▂▇▃▂█
seed_456_episode,▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▇▇▇███
seed_456_reward,▂▃▁▁▃▁▃▁▂▂▂▃▃▆▃▃▆▃▇▅▄█▂▇█▅▅▆▆▆▅█▆▄▇▅▆▇▆▅
seed_789_episode,▁▁▁▁▁▁▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇█████
seed_789_reward,▃▁▃▂▁▂▁▁▁▂▁▂▂▃▃█▅▅▅▃▃▄▅▄▄▆▅▆▃▇▆▇▇▅▅▄▅▆▆▄


...

average_reward_across_seeds,▁
seed_101_episode,▁▁▂▂▂▃▃▃▃▃▄▄▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇███
seed_101_reward,▁▁▂▂▄▅▅▅▆▆▅▆▇▅▇▇▅▆▅▇▆▅▇▆▇▆▇▆▇▇▆▆▇▇▆█▇▇▇█
seed_123_episode,▁▁▁▁▂▂▂▂▂▂▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇██
seed_123_reward,▁▁▄▄▄▅▄▇▅▅█▇▆▆▅▅▅▆▃▇▆▅▄▇▇▅▅▃█▆▅▇▆▇▇▄▆▆▇▆
seed_42_episode,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇█████
seed_42_reward,▂▃▁▁▁▅▁▁▁▂▆█▂▅▁▁▂█▁▆▆▂▆▇▂▂▁▆▆█▆▂▂▂█▂▂▇▂▂
seed_456_episode,▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▇▇████
seed_456_reward,▁▁▁▂▄▁▁▄▆▄▄▃▄▆▆▅▇▅▆▅▅▆█▅▇▆▅▆▆▅▇▆▇▆█▇▇▇▅▇
seed_789_episode,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇██
seed_789_reward,▂▁▃▁▂▁▁▁▃▆▅▅▆▆▇▆▆▅▅▅▆▅▆█▅▅▅▅▆▇▇▇█▅▅▅▅▅▅▅


wandb: Agent Starting Run: 1dd40162 with config:
wandb: 	alpha: 0.0584845642617268
wandb: 	gamma: 0.99
wandb: 	n_bins: 20
wandb: 	n_episodes: 1000
wandb: 	seed: 789
wandb: 	temperature: 0.877312972800229


average_reward_across_seeds,▁
seed_101_episode,▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇████
seed_101_reward,▆▂▅▃▁▂▆▂▃▂▁▂▃▄▄▅▅▆▅▇▇▄▅▅▆█▃▇▇▃▄▆▄▅▅▅▄▆▄▆
seed_123_episode,▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▄▄▄▅▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆█████
seed_123_reward,▂▁▁█▁▄▂▁▃▂▃▄▂▂▂▂▄▂▃▇▂▂▃▂▃▃▃▄▃▃▃▅▅▄▃▃▄▅▃▄
seed_42_episode,▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇██
seed_42_reward,▁▁▃▄▂▁▄▃▅▁▁▅▆▄▅▆▃▃▇▇▃▆▃▆▁▅▇▇██▇▇▁▂▇▂▇█▂█
seed_456_episode,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇█
seed_456_reward,▃▁▁▁▁▁▁▁▁▂▁▂▂▁▂▂▁▁▂▁▁▂▃▃▃▁▃▂▂▂▅▂▂▂▆▅▂▂▇█
seed_789_episode,▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇████
seed_789_reward,▁▂▁▂▁▄▂▂▃▂▂▃▄▅▂▄▆▅▂▅▆▆▄▃█▅▂▁▅▅▂█▃▇▂▃▅▃▆▂


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 6ruirnka with config:
wandb: 	alpha: 0.3382421726811943
wandb: 	gamma: 0.99
wandb: 	n_bins: 20
wandb: 	n_episodes: 1000
wandb: 	seed: 456
wandb: 	temperature: 1.5236135300583142


average_reward_across_seeds,▁
seed_101_episode,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇██
seed_101_reward,▁▁▁▁▅▃▅▅▃▅▅▄▆▅▆▆▆▆▄▆▆▇▆▆▆▆▆▇▆▆▆▇▇▄▆▇█▆▇▆
seed_123_episode,▁▁▁▁▁▁▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇████
seed_123_reward,▄▁▂▃▅▅▇▅▅▅▅▄▄▆▆▄▅▅▆▆▅▇█▆▇▆▇█▆▆▆▇█▇▇▇▆▇▇█
seed_42_episode,▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇████
seed_42_reward,▁▄▄▅▅▂▆▅▆▅▆▆▃▆▆▇▆▆▆▅▃▂▇▇▂▃▂▇▇▇▇▃▂█▇▇▇▇▂▂
seed_456_episode,▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇██████
seed_456_reward,▁▁▄▅▃▃▃▄▃▃▂▂▃▂▄▄▄▄▄▄▅▃▅▆▆▆▄▃▅▆▄▄▆▇█▆▆▃▆▃
seed_789_episode,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇██
seed_789_reward,▂▁▄▅▅▄▆▄▄▇▆▅▅▅▅▆▇▆▅▆▅▄▆▅▅▇▅█▆▇▆▆▇▃▄▆▂█▇█


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import os
import gymnasium as gym

# Function to run SARSA for n_episodes using the existing agent classes
def run_sarsa(env, n_episodes, seeds, alpha, gamma, epsilon, n_bins):
    all_rewards = []

    for seed in seeds:
        np.random.seed(seed)
        env.reset(seed=seed)

        # Initialize agent with the given hyperparameters
        state_discretizer = StateDiscretizer(n_bins=n_bins)
        agent = SARSAAgent(
            state_discretizer=state_discretizer,
            action_space=env.action_space,
            alpha=alpha,
            gamma=gamma,
            epsilon=epsilon
        )

        episode_rewards = []

        for episode in range(n_episodes):
            state, _ = env.reset(seed=seed+episode)
            action = agent.select_action(state)
            done = False
            total_reward = 0

            while not done:
                next_state, reward, terminated, truncated, _ = env.step(action)
                done = terminated or truncated
                total_reward += reward

                next_action = agent.select_action(next_state)
                agent.update(state, action, reward, next_state, next_action, done)

                state = next_state
                action = next_action

            episode_rewards.append(total_reward)

            # Print progress every 1000 episodes
            if (episode + 1) % 1000 == 0:
                print(f"SARSA - Seed {seed}, Episode {episode + 1}/{n_episodes}, Reward: {total_reward}")

        all_rewards.append(episode_rewards)

    return np.array(all_rewards)

# Function to run Q-Learning for n_episodes using the existing agent classes
def run_qlearning(env, n_episodes, seeds, alpha, gamma, temperature, n_bins):
    all_rewards = []

    for seed in seeds:
        np.random.seed(seed)
        env.reset(seed=seed)

        # Initialize agent with the given hyperparameters
        state_discretizer = StateDiscretizer(n_bins=n_bins)
        agent = QLearningAgent(
            state_discretizer=state_discretizer,
            action_space=env.action_space,
            alpha=alpha,
            gamma=gamma,
            temperature=temperature
        )

        episode_rewards = []

        for episode in range(n_episodes):
            state, _ = env.reset(seed=seed+episode)
            done = False
            total_reward = 0

            while not done:
                action = agent.select_action(state)
                next_state, reward, terminated, truncated, _ = env.step(action)
                done = terminated or truncated
                total_reward += reward

                agent.update(state, action, reward, next_state, done)

                state = next_state

            episode_rewards.append(total_reward)

            # Print progress every 1000 episodes
            if (episode + 1) % 1000 == 0:
                print(f"Q-Learning - Seed {seed}, Episode {episode + 1}/{n_episodes}, Reward: {total_reward}")

        all_rewards.append(episode_rewards)

    return np.array(all_rewards)

# Functions for plotting
def plot_results(rewards, title, filename):
    mean_rewards = np.mean(rewards, axis=0)
    std_rewards = np.std(rewards, axis=0)
    episodes = np.arange(1, len(mean_rewards) + 1)

    plt.figure(figsize=(10, 6))
    plt.plot(episodes, mean_rewards, label='Mean Reward')
    plt.fill_between(episodes, mean_rewards - std_rewards, mean_rewards + std_rewards, alpha=0.3, label='Standard Deviation')

    plt.title(title)
    plt.xlabel('Episode')
    plt.ylabel('Reward')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.savefig(filename)
    plt.close()

def plot_comparative_results(sarsa_rewards, qlearning_rewards, filename):
    sarsa_mean = np.mean(sarsa_rewards, axis=0)
    sarsa_std = np.std(sarsa_rewards, axis=0)
    qlearning_mean = np.mean(qlearning_rewards, axis=0)
    qlearning_std = np.std(qlearning_rewards, axis=0)
    episodes = np.arange(1, len(sarsa_mean) + 1)

    plt.figure(figsize=(12, 7))
    plt.plot(episodes, sarsa_mean, label='SARSA Mean', color='blue')
    plt.fill_between(episodes, sarsa_mean - sarsa_std, sarsa_mean + sarsa_std, alpha=0.2, color='blue')
    plt.plot(episodes, qlearning_mean, label='Q-Learning Mean', color='red')
    plt.fill_between(episodes, qlearning_mean - qlearning_std, qlearning_mean + qlearning_std, alpha=0.2, color='red')

    plt.title('SARSA vs Q-Learning in CartPole-v1')
    plt.xlabel('Episode')
    plt.ylabel('Reward')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.savefig(filename)
    plt.close()

# Main function
def main():
    # Create directory for results
    os.makedirs('results', exist_ok=True)

    # Initialize environment
    env = gym.make("CartPole-v1")

    # Set parameters
    n_episodes = 10000
    seeds = [42, 123, 456, 789, 101]  # 5 random seeds

    # Set best hyperparameters from wandb sweeps
    sarsa_params = {
        'alpha': 0.38074,  # Replace with your best value
        'gamma': 0.99,
        'epsilon': 0.15554,  # Replace with your best value
        'n_bins': 10 # Replace with your best value
    }

    qlearning_params = {
        'alpha': 0.2363,  # Replace with best value
        'gamma': 0.99,
        'temperature': 1.89753,  # Replace with best value
        'n_bins': 20  # Replace with best value
    }

    print("Starting SARSA training...")
    sarsa_rewards = run_sarsa(env, n_episodes, seeds, **sarsa_params)
    np.save("results/sarsa_rewards.npy", sarsa_rewards)

    print("Starting Q-Learning training...")
    qlearning_rewards = run_qlearning(env, n_episodes, seeds, **qlearning_params)
    np.save("results/qlearning_rewards.npy", qlearning_rewards)

    # Generate plots
    print("Generating plots...")
    plot_results(sarsa_rewards, 'SARSA in CartPole-v1', 'results/sarsa_plot.png')
    plot_results(qlearning_rewards, 'Q-Learning in CartPole-v1', 'results/qlearning_plot.png')
    plot_comparative_results(sarsa_rewards, qlearning_rewards, 'results/comparative_plot.png')

    print("Done! Results saved in the 'results' directory.")

if __name__ == "__main__":
    main()


Starting SARSA training...
SARSA - Seed 42, Episode 1000/10000, Reward: 27.0
SARSA - Seed 42, Episode 2000/10000, Reward: 114.0
SARSA - Seed 42, Episode 3000/10000, Reward: 174.0
SARSA - Seed 42, Episode 4000/10000, Reward: 185.0
SARSA - Seed 42, Episode 5000/10000, Reward: 13.0
SARSA - Seed 42, Episode 6000/10000, Reward: 47.0
SARSA - Seed 42, Episode 7000/10000, Reward: 182.0
SARSA - Seed 42, Episode 8000/10000, Reward: 121.0
SARSA - Seed 42, Episode 9000/10000, Reward: 184.0
SARSA - Seed 42, Episode 10000/10000, Reward: 208.0
SARSA - Seed 123, Episode 1000/10000, Reward: 174.0
SARSA - Seed 123, Episode 2000/10000, Reward: 119.0
SARSA - Seed 123, Episode 3000/10000, Reward: 45.0
SARSA - Seed 123, Episode 4000/10000, Reward: 112.0
SARSA - Seed 123, Episode 5000/10000, Reward: 66.0
SARSA - Seed 123, Episode 6000/10000, Reward: 225.0
SARSA - Seed 123, Episode 7000/10000, Reward: 261.0
SARSA - Seed 123, Episode 8000/10000, Reward: 203.0
SARSA - Seed 123, Episode 9000/10000, Reward: 170.0